WIP example of using `any_expr` to generate sklearn code


In [1]:
import os

os.environ["SCIPY_ARRAY_API"] = "1"

from sklearn import set_config
from sklearn.datasets import make_classification
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from egglog import *
from egglog.exp import any_expr
from egglog.exp.any_expr import *

set_config(array_api_dispatch=True)

_0 = list((*(), 42))
del _0[0]
assert (list((*(), 42)) != 10)
assert (not (_0 == 1))
-_0


In [2]:
@ruleset
def extra_rules(x: A, y: A, z: A, o: PyObject):
    yield rewrite(x.__array_namespace__(y, z).__name__, subsume=True).to(A(__name__))

    #     File ~/p/egg-smol-python/.venv/lib/python3.13/site-packages/sklearn/utils/_unique.py:72, in _cached_unique(y, xp)
    #      63 """Return the unique values of y.
    #      64
    #      65 Use the cached values from dtype.metadata if present.
    #    (...)     69 Call `attach_unique` to attach the unique values to y.
    #      70 """
    #      71 try:
    # ---> 72     if y.dtype.metadata is not None and "unique" in y.dtype.metadata:

    yield rule(eq(A(None)).to(x.dtype.metadata)).then(getattr_eager(x.dtype, "metadata"))

    # File ~/p/egg-smol-python/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:1093, in check_array(array, accept_sparse, accept_large_sparse, dtype, order, copy, force_writeable, force_all_finite, ensure_all_finite, ensure_non_negative, ensure_2d, allow_nd, ensure_min_samples, ensure_min_features, estimator, input_name)
    #    1085             msg = (
    #    1086                 f"Expected 2D array, got 1D array instead:\narray={array}.\n"
    #    1087                 "Reshape your data either using array.reshape(-1, 1) if "
    #    1088                 "your data has a single feature or array.reshape(1, -1) "
    #    1089                 "if it contains a single sample."
    #    1090             )
    #    1091         raise ValueError(msg)
    # -> 1093 if dtype_numeric and hasattr(array.dtype, "kind") and array.dtype.kind in "USV":
    #    1094     raise ValueError(
    #    1095         "dtype='numeric' is not compatible with arrays of bytes/strings."
    #    1096         "Convert your data to numeric values explicitly instead."
    #    1097     )
    #    1098 if not allow_nd and array.ndim >= 3:

    # TypeError: 'in <string>' requires string as left operand, not RuntimeExpr
    # Eagerly eval kind on dtype attribute
    yield rule(x.dtype.kind).then(getattr_eager(x.dtype, "kind"))


any_expr.any_expr_schedule = (extra_rules.saturate() + any_expr_ruleset).saturate()

In [3]:
lda = LinearDiscriminantAnalysis()
X_np, y_np = make_classification(random_state=0)

egraph = EGraph()
with set_any_expr_egraph(egraph):
    x = lda.fit_transform(AnyExpr(X_np), AnyExpr(y_np))

In [4]:
# print(str(x))

In [5]:
print(any_expr_source(x))

_0 = array([[-0.03926799,  0.13191176, -0.21120598, ...,  1.97698901,
         1.02122474, -0.46931074],
       [ 0.77416061,  0.10490717, -0.33281176, ...,  1.2678044 ,
         0.62251914, -1.49026539],
       [-0.0148577 ,  0.67057045, -0.21416666, ..., -0.10486202,
        -0.10169727, -0.45130304],
       ...,
       [ 0.29673317, -0.49610233, -0.86404499, ..., -1.10453952,
         2.01406015,  0.69042902],
       [ 0.08617684,  0.9836362 ,  0.17124355, ...,  2.11564734,
         0.11273794,  1.20985013],
       [-1.58249448, -1.42279491, -0.56430103, ...,  1.26661394,
        -1.31771734,  1.61805427]], shape=(100, 20))
_1 = array([0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0])
_2 = _0.__array_namespac

In [6]:
# egraph = EGraph()
# # x = given(given(10, 100), 1000)
# egraph.register(x)
# egraph.run(given_ruleset.saturate())
# # egraph.display()
# print(str(egraph.extract(x)))  # $, cost_model=greedy_dag_cost_model())))

# a

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
